In [1]:
import pandas
print(pandas.__version__)


2.2.3


SyntaxError: invalid syntax (955020236.py, line 1)

In [2]:
import osmnx as ox
import networkx as nx
import numpy as np
from geopy.geocoders import Nominatim
from scipy.optimize import linear_sum_assignment
import time
import folium
from folium.plugins import MarkerCluster

In [3]:
pip install matplotlib

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install osmnx networkx

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
pip install streamlit


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.6 MB 1.9 MB/s eta 0:00:05
   --- ------------------------------------ 0.8/9.6 MB 1.8 MB/s eta 0:00:05
   ----- ---------------------------------- 1.3/9.6 MB 1.9 MB/s eta 0:00:05
   ----- ---------------------------------- 1.3/9.6 MB 1.9 MB/s eta 0:00:05
   ------ --------------------------------- 1.6/9.6 MB 1.6 MB/s eta 0:00:06
   ------ --------------------------------- 1.6/9.6 MB 1.6 MB/s eta 0:00:06
   ------- -------------------------------- 1.8/9.6 MB 1.2 MB/s eta 0:00:07
   ------- -------------------------------- 1.8/9.6 MB 1.2 MB/s eta 0:00:07
   --------- ------------------------------ 2.4/9.6 MB 1.1 MB/s eta 0:00:07
   ---------- --------------------


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
# Install required libraries
!pip install osmnx geopy scipy matplotlib --quiet



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
pip install ortools


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/18.1 MB ? eta -:--:--
   - -------------------------------------- 0.8/18.1 MB 3.7 MB/s eta 0:00:05
   --- ------------------------------------ 1.6/18.1 MB 3.9 MB/s eta 0:00:05
   ---- ----------------------------------- 2.1/18.1 MB 3.6 MB/s eta 0:00:05
   ------ --------------------------------- 2.9/18.1 MB 3.4 MB/s eta 0:00:05
   -------- ------------------------------- 3.7/18.1 MB 3.6 MB/s eta 0:00:05
   --------- ------------------------------ 4.2/18.1 MB 3.5 MB/s eta 0:00:04
   --------- ------------------------------ 4.5/18.1 MB 3.3 MB/s eta 0:00:05
   ----------- ---------------------------- 5.2/18.1 MB 3.2 MB/s eta 0:00:05
   ------------ --------------------------- 5.8/18.1 MB 3.1 MB/s eta 0:00:04
   ------------- -------------------------- 6.0/18.1 MB 3.0 MB/s eta 0:00:04
   -------------- ------------------------- 6.6/18.1 MB 2.8 MB/s eta 0:00:05
   --


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install folium
!pip install requests

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import requests
API_KEY = "M4gbWbXRcVHKsmy42AesQzR2rlrUarfm"  # Replace with your actual API key

In [2]:
!jupyter nbconvert --to python AdvancedVRP.ipynb


This application is used to convert notebook files (*.ipynb)

[NbConvertApp] WARNING | pattern 'AdvancedVRP.ipynb' matched no files



        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
    Execute the notebook prior to export.
    Equivalent to: [--ExecutePreprocessor.enabled=True]
--allow-errors
    Continue noteboo

In [ ]:
import os
import time
import random
import requests
import folium
import itertools
import numpy as np
import networkx as nx
import osmnx as ox
from folium.plugins import MarkerCluster, Search, AntPath
from folium.features import CustomIcon, DivIcon
from shapely.geometry import LineString
from networkx.algorithms import approximation as approx
from geopy.geocoders import Nominatim

# =========== NEW IMPORTS FOR OR-TOOLS =============
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
# ==================================================

########################################
# Helper: Force conversion to Python int
########################################
def to_python_int(x):
    try:
        return int(x)
    except Exception:
        return int(x.item())

########################################
# 0) Fallbacks (if needed)
########################################
geo_fallbacks = {
    "Warehouse 1, Jabal Ali, Dubai, UAE": (24.9837, 55.0673),
    "Warehouse 2, Al Awir, Dubai, UAE": (25.1555, 55.5045),
    "Fulfillment Center 1, Mudon, Dubai, UAE": (25.0529, 55.2774),
    "Fulfillment Center 2, Al Manara, Dubai, UAE": (25.1412, 55.2148),
    "Fulfillment Center 3, Al Garhoud, Dubai, UAE": (25.2432, 55.3487),
    "Fulfillment Center 4, Warsan 1, Dubai, UAE": (25.1614, 55.4184),
}

########################################
# 1) Setup & Geocoding
########################################
geolocator = Nominatim(user_agent="route_optimization_app")
def geocode_locations(locations_list):
    geocoded = []
    for loc in locations_list:
        try:
            r = geolocator.geocode(loc, timeout=5)
            if r:
                geocoded.append((r.latitude, r.longitude))
                print(f"✅ {loc} => {r.latitude},{r.longitude}")
            else:
                if loc in geo_fallbacks:
                    fallback = geo_fallbacks[loc]
                    geocoded.append(fallback)
                    print(f"⚠️ Using fallback for {loc}")
                else:
                    geocoded.append(None)
                    print(f"❌ {loc} => no result")
        except Exception as e:
            geocoded.append(None)
            print(f"❌ Error geocoding {loc}: {e}")
        time.sleep(0.1)
    return geocoded

########################################
# 2) TomTom
########################################
def fetch_tomtom_traffic(coords, tomtom_key):
    base_url = "https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json"
    speeds = {}
    for (lat, lon) in coords:
        params = {"point": f"{lat},{lon}", "unit": "KMPH", "key": tomtom_key}
        try:
            r = requests.get(base_url, params=params)
            r.raise_for_status()
            data = r.json()
            if "flowSegmentData" in data:
                speeds[(lat,lon)] = data["flowSegmentData"].get("currentSpeed", 1)
            else:
                speeds[(lat,lon)] = 1
        except:
            speeds[(lat,lon)] = 1
    print(f"✅ TomTom traffic => {len(speeds)} coords")
    return speeds

########################################
# 3) All Locations
########################################
all_locations = [
    # 2 warehouses
    "Warehouse 1, Jabal Ali, Dubai, UAE",   # index 0
    "Warehouse 2, Al Awir, Dubai, UAE",       # index 1
    # 4 FC
    "Fulfillment Center 1, Mudon, Dubai, UAE",  # index 2
    "Fulfillment Center 2, Al Manara, Dubai, UAE",  # index 3
    "Fulfillment Center 3, Al Garhoud, Dubai, UAE", # index 4
    "Fulfillment Center 4, Warsan 1, Dubai, UAE", # index 5
    # 20 deliveries => 6..25
    "Arabian Ranches 2, Dubai, UAE",
    "Remraam, Dubai, UAE",
    "DAMAC Hills, Dubai, UAE",
    "Dubai Studio City, Dubai, UAE",
    "Motor City, Dubai, UAE",
    "Jumeirah Beach, Dubai, UAE",
    "Umm Suqeim, Dubai, UAE",
    "Al Safa, Dubai, UAE",
    "Al Wasl, Dubai, UAE",
    "Al Barsha, Dubai, UAE",
    "Port Saeed, Dubai, UAE",
    "Umm Ramool, Dubai, UAE",
    "Nad Al Hamar, Dubai, UAE",
    "Mirdif, Dubai, UAE",
    "Al Rashidiya, Dubai, UAE",
    "International City, Dubai, UAE",
    "Academic City, Dubai, UAE",
    "Nad Al Sheba, Dubai, UAE",
    "Al Warqa, Dubai, UAE",
    "Dubai Silicon Oasis, Dubai, UAE",
]

# For deliveries
time_priority_info = {}
for i, loc in enumerate(all_locations):
    if i < 6:
        time_priority_info[loc] = {"priority": None, "time_window": None}
    else:
        pr = random.randint(1, 3)
        s = random.randint(8, 12)
        e = s + random.randint(2, 4)
        time_priority_info[loc] = {"priority": pr, "time_window": (s, e)}

########################################
# 4) Build and Preprocess OSMnx Graph
########################################
print("Downloading OSMnx graph for 'Dubai, United Arab Emirates'...")
graph = ox.graph_from_place("Dubai, United Arab Emirates", network_type="drive")
print("OSMnx graph => done")
# Convert to undirected and limit to largest connected component.
graph = graph.to_undirected()
largest_cc = max(nx.connected_components(graph), key=len)
graph = graph.subgraph(largest_cc).copy()
print("Graph preprocessed: converted to undirected and limited to largest connected component.")

########################################
# 5) Fuel & Emission
########################################
def calculate_fuel_and_emissions(dist_km, fe=12.0, prc=3.0, co2f=0.2):
    cost = (dist_km / fe) * prc
    co2  = dist_km * co2f
    return cost, co2

########################################
# 6) Update with TomTom
########################################
def update_graph_with_tomtom(g, speeds):
    from shapely.geometry import LineString
    for u, v, k, data in g.edges(keys=True, data=True):
        if "geometry" in data and isinstance(data["geometry"], LineString):
            lat, lon = data["geometry"].coords[0]
        else:
            lat, lon = None, None
        if (lat, lon) in speeds:
            try:
                spd = speeds[(lat, lon)]
                if spd > 0:
                    t = data["length"] / (spd * 1000/3600)
                    data["travel_time"] = t * 1.2
                else:
                    data["travel_time"] = float("inf")
            except:
                pass
    print("Graph updated with TomTom speeds.")
    return g

########################################
# 7) Drivers array
########################################
drivers = [
    {"id": 1, "color": "red"},
    {"id": 2, "color": "blue"},
    {"id": 3, "color": "green"},
    {"id": 4, "color": "purple"},
    {"id": 5, "color": "orange"},
    {"id": 6, "color": "darkred"},
    {"id": 7, "color": "darkblue"},
    {"id": 8, "color": "darkgreen"},
    {"id": 9, "color": "cadetblue"},
    {"id": 10, "color": "black"},
]

########################################
# 8) Partition deliveries by nearest FC
########################################
def partition_deliveries_by_fc(node_list):
    fc_idx = [2, 3, 4, 5]
    groups = {fc: [] for fc in fc_idx}
    for i in range(6, len(all_locations)):
        best_fc = None
        best_d = float('inf')
        for fc in fc_idx:
            try:
                dd = nx.shortest_path_length(graph, node_list[fc], node_list[i], weight="length")
                if dd < best_d:
                    best_d = dd
                    best_fc = fc
            except Exception:
                continue
        if best_fc is not None:
            groups[best_fc].append(i)
    return groups

########################################
# 9) Sub VRP (TSP version: single vehicle per FC group)
########################################
def solve_sub_vrp(fc, deliveries, node_list, G):
    if not deliveries:
        return []
    # Create a local list of global node IDs for the subproblem.
    local_nodes = [node_list[fc]] + [node_list[i] for i in deliveries]
    n = len(local_nodes)
    # Mapping: local index 0 corresponds to fc; local indices 1...n-1 correspond to deliveries.
    local_to_global = {0: fc}
    for i, d in enumerate(deliveries, start=1):
        local_to_global[i] = d

    # Build the cost matrix using the local node IDs.
    cost_mat = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if i == j:
                cost_mat[i][j] = 0
            else:
                try:
                    d = nx.shortest_path_length(G, local_nodes[i], local_nodes[j], weight="length")
                    cost_mat[i][j] = d
                except:
                    cost_mat[i][j] = 9999999

    # For a TSP, we use a single vehicle.
    num_veh = 1
    data = {
        'distance_matrix': cost_mat,
        'depot': 0
    }

    # Create the RoutingIndexManager using local indices.
    manager = pywrapcp.RoutingIndexManager(n, num_veh, data['depot'])
    routing = pywrapcp.RoutingModel(manager)

    def transit_cb(from_index, to_index):
        f = to_python_int(from_index)
        t = to_python_int(to_index)
        i = int(manager.IndexToNode(f))
        j = int(manager.IndexToNode(t))
        return data['distance_matrix'][i][j]

    tcb_idx = routing.RegisterTransitCallback(transit_cb)
    routing.SetArcCostEvaluatorOfAllVehicles(tcb_idx)

    # For a TSP, we do not need a demand callback.
    search_params = pywrapcp.DefaultRoutingSearchParameters()
    search_params.time_limit.seconds = 120
    search_params.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

    assignment = routing.SolveWithParameters(search_params)
    if not assignment:
        print(f"❌ No sub-solution for FC={fc}, group={deliveries}")
        return []

    route = []
    index = routing.Start(0)
    while not routing.IsEnd(index):
        route.append(int(manager.IndexToNode(to_python_int(index))))
        index = assignment.Value(routing.NextVar(index))
    route.append(int(manager.IndexToNode(to_python_int(index))))
    # Map local indices back to original location indices.
    global_route = [local_to_global[i] for i in route]
    # Only keep the route if it visits at least one delivery.
    if len(global_route) > 2:
        return [global_route]
    return []

########################################
# 10) Solve Overall VRP
########################################
def solve_overall_vrp(node_list, G):
    groups = partition_deliveries_by_fc(node_list)
    overall = []
    for fc, dlist in groups.items():
        if not dlist:
            continue
        sub_sol = solve_sub_vrp(fc, dlist, node_list, G)
        overall.extend(sub_sol)
    return overall

########################################
# 11) Baseline
########################################
def calculate_naive_baseline(geocoded, node_list, G):
    fc_idx = [2, 3, 4, 5]
    dist = 0.0
    fuel = 0.0
    co2 = 0.0
    for i in range(6, len(all_locations)):
        best = None
        for fc in fc_idx:
            try:
                outp = nx.shortest_path(G, node_list[fc], node_list[i], weight="length")
                retp = nx.shortest_path(G, node_list[i], node_list[fc], weight="length")
                dd_out = sum(G.get_edge_data(outp[k], outp[k+1], 0).get("length", 0) for k in range(len(outp)-1))
                dd_ret = sum(G.get_edge_data(retp[k], retp[k+1], 0).get("length", 0) for k in range(len(retp)-1))
                tot_km = (dd_out + dd_ret) / 1000.0
                if best is None or tot_km < best:
                    best = tot_km
            except:
                continue
        if best is not None:
            fcst, co2v = calculate_fuel_and_emissions(best)
            dist += best
            fuel += fcst
            co2 += co2v
    return dist, fuel, co2

########################################
# 12) run_vrp
########################################
def run_vrp():
    # 1) Geocode locations
    geocoded = geocode_locations(all_locations)
    node_list = []
    for c in geocoded:
        if c is None:
            node_list.append(None)
        else:
            # Nominatim returns (lat, lon); nearest_nodes expects (lon, lat)
            n = ox.distance.nearest_nodes(graph, c[1], c[0])
            node_list.append(n)
    if any(n is None for n in node_list):
        print("❌ Some location not resolved => exit")
        return (0, 0, 0, 0)

    # 2) TomTom Traffic
    key = "M4gbWbXRcVHKsmy42AesQzR2rlrUarfm"
    speeds = fetch_tomtom_traffic(geocoded, key)

    # 3) Update Graph
    g_up = update_graph_with_tomtom(graph, speeds)

    # 4) Baseline Calculation
    base_dist, base_fuel, base_co2 = calculate_naive_baseline(geocoded, node_list, g_up)

    # 5) Solve Overall VRP
    all_routes = solve_overall_vrp(node_list, g_up)
    if not all_routes:
        print("❌ No overall routes => no result.")
        return (0, 0, 0, 0)

    # Assign drivers round-robin
    route_assignments = []
    for i, rt in enumerate(all_routes):
        driver = drivers[i % len(drivers)]
        route_assignments.append({"driver": driver, "route": rt})

    # 6) Build Folium Map
    m = folium.Map(tiles="CartoDB Positron", zoom_start=10)
    m.fit_bounds([(c[0], c[1]) for c in geocoded if c])
    mk_fg = folium.FeatureGroup("Markers").add_to(m)
    vr_fg = folium.FeatureGroup("Multi-Driver VRP").add_to(m)

    def wh_divicon(lbl):
        return DivIcon(icon_size=(50, 50), icon_anchor=(25, 50),
                       html=f'<div style="text-align:center;font-size:24px;">🏭<br/><span style="font-size:12px;color:darkblue;">{lbl}</span></div>')
    def fc_divicon(lbl):
        return DivIcon(icon_size=(50, 50), icon_anchor=(25, 50),
                       html=f'<div style="text-align:center;font-size:24px;">📦<br/><span style="font-size:12px;color:darkgreen;">{lbl}</span></div>')
    def deliv_pin():
        return DivIcon(icon_size=(24, 24), icon_anchor=(12, 24),
                       html='<div style="font-size:24px;text-align:center;">📍</div>')

    fc_conn_txt = {2: "FC2 & FC4", 3: "FC1 & FC3", 4: "FC2 & FC4", 5: "FC3 & FC1"}

    # Place markers for each location
    deliv_count = 1
    for idx, (loc, gc) in enumerate(zip(all_locations, geocoded)):
        if not gc:
            continue
        lat, gc_lon = gc
        if idx < 2:
            lbl = f"Warehouse {idx+1}"
            cat = "FC1 & FC2" if idx == 0 else "FC3 & FC4"
            pop = f"<b>{loc}</b><br/>Lat:{lat:.5f}, Lon:{gc_lon:.5f}<br/>{cat}"
            folium.Marker((lat, gc_lon), tooltip=lbl, popup=pop, icon=wh_divicon(lbl)).add_to(mk_fg)
        elif idx < 6:
            fc_lbl = f"Fulfillment Center (FC{idx-1})"
            pop = f"<b>{loc}</b><br/>Lat:{lat:.5f}, Lon:{gc_lon:.5f}<br/>{fc_conn_txt.get(idx, '')}"
            folium.Marker((lat, gc_lon), tooltip=fc_lbl, popup=pop, icon=fc_divicon(fc_lbl)).add_to(mk_fg)
        else:
            pr = time_priority_info[loc]["priority"]
            ttip = f"Delivery {deliv_count}"
            pop = f"<b>{loc}</b><br/>Delivery {deliv_count}<br/>Lat:{lat:.5f}, Lon:{gc_lon:.5f}<br/>Priority:{pr}"
            folium.Marker((lat, gc_lon), tooltip=ttip, popup=pop, icon=deliv_pin()).add_to(mk_fg)
            deliv_count += 1

    # Draw FC-FC and Warehouse-FC connecting lines
    fc_pairs = [(2, 3), (3, 4), (4, 5), (5, 2)]
    for (f1, f2) in fc_pairs:
        if geocoded[f1] and geocoded[f2]:
            folium.PolyLine([geocoded[f1], geocoded[f2]], color="darkcyan", weight=3, dash_array="5,5",
                            tooltip=f"{all_locations[f1]} ↔ {all_locations[f2]}").add_to(vr_fg)
    wh_fc = [(0, 2), (0, 3), (1, 4), (1, 5)]
    for (w, f) in wh_fc:
        if geocoded[w] and geocoded[f]:
            folium.PolyLine([geocoded[w], geocoded[f]], color="teal", weight=3, dash_array="3,6",
                            tooltip=f"{all_locations[w]} ↔ {all_locations[f]}").add_to(vr_fg)

    # Summations and routing details
    total_dist = 0.0
    total_fuel = 0.0
    total_co2 = 0.0
    sum_time = 0.0
    num_deliv_legs = 0
    base_speed = 40.0

    def count_signals(path_nodes, G):
        return sum(1 for n in path_nodes if G.nodes[n].get("highway") == "traffic_signals")

    fc_label_map = {2: "FC1", 3: "FC2", 4: "FC3", 5: "FC4"}

    for info in route_assignments:
        route = info["route"]
        if len(route) < 2:
            continue
        path_nodes = []
        dist_m = 0.0
        for i in range(len(route) - 1):
            try:
                sp = nx.shortest_path(g_up, node_list[route[i]], node_list[route[i+1]], weight="length")
                dist_m += sum(g_up.get_edge_data(sp[k], sp[k+1], 0).get("length", 0) for k in range(len(sp)-1))
                path_nodes.extend(sp)
            except:
                continue
        dist_km = dist_m / 1000.0
        f_, c_ = calculate_fuel_and_emissions(dist_km)
        total_dist += dist_km
        total_fuel += f_
        total_co2 += c_

        sigs = count_signals(path_nodes, g_up)
        full_time = (dist_km / base_speed) * 60 + (sigs * 0.5)

        for i in range(len(route) - 2):
            try:
                sp = nx.shortest_path(g_up, node_list[route[i]], node_list[route[i+1]], weight="length")
                dd = sum(g_up.get_edge_data(sp[k], sp[k+1], 0).get("length", 0) for k in range(len(sp)-1))
                s_ = count_signals(sp, g_up)
                leg_time = (dd / 1000.0 / base_speed) * 60 + (s_ * 0.5)
                sum_time += leg_time
                num_deliv_legs += 1
            except:
                continue

        for i in range(len(route) - 1):
            try:
                sp = nx.shortest_path(g_up, node_list[route[i]], node_list[route[i+1]], weight="length")
                ddm = sum(g_up.get_edge_data(sp[k], sp[k+1], 0).get("length", 0) for k in range(len(sp)-1))
                s_ = count_signals(sp, g_up)
                leg_t = (ddm / 1000.0 / base_speed) * 60 + (s_ * 0.5)
                if i == len(route) - 2 and route[i+1] in fc_label_map:
                    leg_str = f"Back to {fc_label_map[route[i+1]]}"
                elif i == len(route) - 2:
                    leg_str = "Back to FC"
                else:
                    leg_str = f"Leg {i+1}"
                tip = (f"Driver {info['driver']['id']}, {leg_str}<br/>"
                       f"Leg: {round(ddm/1000.0,2)} km, {round(leg_t,2)} min")
                coords = [(g_up.nodes[x]["y"], g_up.nodes[x]["x"]) for x in sp]
                AntPath(locations=coords, dash_array=[10,20], delay=600,
                        color=info["driver"]["color"], weight=4, tooltip=tip).add_to(vr_fg)
            except:
                continue

    avg_time = round(sum_time/num_deliv_legs, 2) if num_deliv_legs > 0 else 0.0

    folium.LayerControl().add_to(m)
    Search(layer=mk_fg, search_label='tooltip', placeholder='Search...', collapsed=False).add_to(m)

    legend_html = '''
    <div style="position: fixed; bottom: 30px; left: 30px; width: 220px;
         background-color: rgba(255,255,255,0.9);
         z-index:9999; border:2px solid #444; border-radius:5px; padding:10px;
         font-size:12px;">
      <h4 style="margin-top:0;">Legend</h4>
      <b>Driver Routes</b><br/>
      <span style="color:red;">■</span> D1 &nbsp;
      <span style="color:blue;">■</span> D2 &nbsp;
      <span style="color:green;">■</span> D3 &nbsp;
      <span style="color:purple;">■</span> D4 <br/>
      <span style="color:orange;">■</span> D5 &nbsp;
      <span style="color:darkred;">■</span> D6 &nbsp;
      <span style="color:darkblue;">■</span> D7 &nbsp;
      <span style="color:darkgreen;">■</span> D8 <br/>
      <span style="color:cadetblue;">■</span> D9 &nbsp;
      <span style="color:black;">■</span> D10 <br/>
      <hr style="margin:5px 0;">
      <b>Icons</b><br/>
      🏭 Warehouses<br/>
      📦 Fulfillment Centers<br/>
      📍 Delivery Points<br/>
      <hr style="margin:5px 0;">
      <small>FC-FC: darkcyan dotted lines<br/>
      WH-FC: teal dashed lines</small>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))

    co2_saved = round(base_co2 - total_co2, 2)
    fuel_saved = round(base_fuel - total_fuel, 2)
    disclaim_html = f'''
    <div style="position: fixed; bottom: 30px; right: 30px; width: 320px;
         background-color: rgba(255,255,255,0.9);
         z-index:9999; border:2px solid #444; border-radius:5px; padding:10px;
         font-size:12px; line-height:1.3em;">
      <b>Map Overview</b><br/>
      * Deliveries are partitioned by nearest FC.<br/>
      * Each sub-VRP uses a single vehicle route (TSP).<br/>
      * No route distance limit => no OverflowError.<br/>
      <hr>
      <b>🌱 Sustainability Impact</b><br/>
      ♻️ CO₂ Saved: {co2_saved} kg<br/>
      ⛽ Fuel Cost Saved: AED {fuel_saved}<br/>
      <hr>
      <b>Avg Delivery Time (excl. final leg)</b><br/>
      {avg_time} min
    </div>
    '''
    m.get_root().html.add_child(folium.Element(disclaim_html))

    hover_js = """
    <script>
      var all_elems = document.getElementsByClassName('leaflet-interactive');
      for(var i=0; i<all_elems.length; i++){
          all_elems[i].addEventListener('mouseover', function(ev){
              ev.target.setAttribute('stroke-width','6');
          });
          all_elems[i].addEventListener('mouseout', function(ev){
              ev.target.setAttribute('stroke-width','4');
          });
      }
    </script>
    """
    m.get_root().html.add_child(folium.Element(hover_js))

    m.save("ExpandedDubai_VRP.html")
    print("✅ Final expanded VRP => ExpandedDubai_VRP.html")
    return (round(total_dist, 2), round(total_co2, 2), round(total_fuel, 2), avg_time)

if __name__ == "__main__":
    run_vrp()